In [1]:
import os
import sys
import geopandas as gpd
import pandas as pd
import requests
import time
import xarray as xr
import numpy as np
import tempfile
import contextlib
import rasterio
from rasterio.transform import from_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
from datetime import datetime, timedelta, timezone
from collections import deque
from osgeo import gdal, osr
import warnings
warnings.filterwarnings('ignore')

# GDAL 설정
gdal.UseExceptions()

# 설정
date_column = 'date'
latitude_column = 'y'
longitude_column = 'x'
SERVICE_KEY = os.environ.get("KMA_SERVICE_KEY", "5LGEH7S2SVSxhB-0tnlUlA")
GRIB_API_BASE = "https://apihub.kma.go.kr/api/typ06/url/nwp_vars_down.php"

# 타임존 설정
KST = timezone(timedelta(hours=9))
USE_KST = True

# 좌표계 설정
TARGET_CRS = "EPSG:5179"  # Korea 2000 / Central Belt
TARGET_RES = 30  # 30m 해상도 (미터 단위)

print("="*70)
print("LDAPS 강우 데이터 처리 파이프라인 (수정 버전)")
print("="*70)
print(f"API 키: {SERVICE_KEY[:10]}...")
print(f"목표 좌표계: {TARGET_CRS} (Korea 2000 / Central Belt)")
print(f"목표 해상도: {TARGET_RES}m")
print("💡 GDAL HDF5 의존성 문제 해결: rasterio 직접 변환 사용")
print("="*70)

LDAPS 강우 데이터 처리 파이프라인 (수정 버전)
API 키: 5LGEH7S2SV...
목표 좌표계: EPSG:5179 (Korea 2000 / Central Belt)
목표 해상도: 30m
💡 GDAL HDF5 의존성 문제 해결: rasterio 직접 변환 사용


In [2]:
# ============================================================
# Step 1 - LDAPS 강우 지표 생성 (Daily TIFFs)
# ============================================================

# 설정
START_DATE = "2019-08-17"
END_DATE = "2020-02-28"
RAW_BASE_DIR = r'D:\Landslide\data\raw\LDAPS'
PROCESSED_BASE_DIR = r'D:\Landslide\data\processed\gyeongnam\LDAPS'

GRIB_DIR = os.path.join(RAW_BASE_DIR, 'GRIB')
NETCDF_DIR = os.path.join(RAW_BASE_DIR, 'NetCDF')
TIF_RAW_DIR = os.path.join(RAW_BASE_DIR, 'TIF_WGS84')
TIF_5179_DIR = os.path.join(PROCESSED_BASE_DIR, 'TIF_5179_30m_20190817_20200228_original')

VARIABLES = ["lspr"]

# 디렉토리 생성
os.makedirs(GRIB_DIR, exist_ok=True)
os.makedirs(NETCDF_DIR, exist_ok=True)
os.makedirs(os.path.join(TIF_RAW_DIR, "acc7d"), exist_ok=True)
os.makedirs(os.path.join(TIF_RAW_DIR, "acc3d"), exist_ok=True)
os.makedirs(os.path.join(TIF_RAW_DIR, "peak1h"), exist_ok=True)
os.makedirs(os.path.join(TIF_5179_DIR, "acc7d"), exist_ok=True)
os.makedirs(os.path.join(TIF_5179_DIR, "acc3d"), exist_ok=True)
os.makedirs(os.path.join(TIF_5179_DIR, "peak1h"), exist_ok=True)

print("\n" + "="*70)
print("Step 1: LDAPS 강우 지표 생성")
print("="*70)
print(f"처리 기간: {START_DATE} ~ {END_DATE}")
print(f"원본 저장 (raw):")
print(f"  - GRIB: {GRIB_DIR}")
print(f"  - NetCDF: {NETCDF_DIR}")
print(f"  - TIF (WGS84): {TIF_RAW_DIR}")
print(f"처리 저장 (processed):")
print(f"  - TIF (5179, 30m): {TIF_5179_DIR}")
print(f"좌표계: WGS84 → {TARGET_CRS}")
print(f"보간법: Cubic (rasterio)")
print(f"해상도: ~1.5km → {TARGET_RES}m")
print("="*70)


def download_grib_file(service_key, base_time, lead_hour, variable, output_dir, max_retries=3, retry_delay=5):
    """
    GRIB 파일을 다운로드하고 파일 경로를 반환
    
    개선 사항:
    - 자동 재시도 로직 (Exponential backoff)
    - 연결 타임아웃 에러 처리
    - 더 긴 타임아웃 설정 (API 서버가 느릴 수 있음)
    """
    url = f'{GRIB_API_BASE}?nwp=l015&sub=unis&vars={variable}&tmfc={base_time}&ef={lead_hour+1}&dataType=GRIB&authKey={service_key}'
    
    filename = f"l015_{base_time}_{variable}_h{lead_hour:03d}.gb2"
    filepath = os.path.join(output_dir, filename)
    
    if os.path.exists(filepath):
        return filepath
    
    print(f"[DOWNLOAD] {filename}")
    
    # 재시도 루프
    for attempt in range(1, max_retries + 1):
        try:
            # 타임아웃을 더 길게 설정 (연결 30초, 읽기 300초)
            r = requests.get(url, timeout=(30, 300), stream=True)
            
            if r.status_code != 200:
                print(f"[ERROR] HTTP {r.status_code} for {filename}")
                return None
                
            content_type = r.headers.get('content-type', '')
            if 'application/octet-stream' not in content_type:
                error_text = r.text[:200]
                if "file not exist" in error_text or "파라미터 없음" in error_text:
                    return None
                
            with open(filepath, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        
            file_size = os.path.getsize(filepath)
            if file_size > 0:
                print(f"[SUCCESS] Downloaded {filename} ({file_size} bytes)")
                return filepath
            else:
                os.remove(filepath)
                return None
            
        except (requests.exceptions.Timeout, 
                requests.exceptions.ConnectionError,
                requests.exceptions.ConnectTimeout) as e:
            # 타임아웃 또는 연결 에러 발생
            if attempt < max_retries:
                wait_time = retry_delay * attempt  # Exponential backoff
                print(f"[RETRY] {filename} - 시도 {attempt}/{max_retries} 실패")
                print(f"        에러: {type(e).__name__}")
                print(f"        {wait_time}초 후 재시도...")
                time.sleep(wait_time)
                continue
            else:
                print(f"[ERROR] 모든 재시도 실패 {filename}: {e}")
                return None
                
        except Exception as e:
            print(f"[ERROR] Failed to download {filename}: {e}")
            return None
    
    return None


def grib_to_xarray(grib_file):
    """GRIB 파일을 xarray Dataset으로 로드"""
    try:
        ds = xr.open_dataset(grib_file, engine='cfgrib')
        return ds
    except Exception as e:
        print(f"[ERROR] Failed to load GRIB {grib_file}: {e}")
        return None


def pick_reference_latlon_from_ds(ds):
    """첫 성공적으로 로드된 GRIB의 위경도 2D를 사용"""
    if 'latitude' in ds.coords and 'longitude' in ds.coords:
        lat = ds['latitude'].values
        lon = ds['longitude'].values
        assert lat.ndim == 2 and lon.ndim == 2, "lat/lon must be 2-D"
        return lat, lon
    
    for cand_lat, cand_lon in [('lat', 'lon'), ('Latitude', 'Longitude')]:
        if cand_lat in ds and cand_lon in ds:
            return ds[cand_lat].values, ds[cand_lon].values
    
    raise RuntimeError("No 2-D lat/lon found in GRIB dataset")


def build_daily_netcdf(day, stack, hours, lat2d, lon2d, out_path):
    """일별 NetCDF 생성 - CF 규약 준수"""
    start = pd.Timestamp(day.date())
    if USE_KST:
        start = start.tz_localize(KST)
        processed_hours = []
        for h in hours:
            if hasattr(h, 'tzinfo') and h.tzinfo is not None:
                processed_hours.append(h.astimezone(KST) if h.tzinfo != KST else h)
            else:
                processed_hours.append(pd.Timestamp(h).tz_localize(KST))
        hours = processed_hours
    
    full_time = pd.date_range(start, start + pd.Timedelta(hours=23), freq="1h", tz=start.tz)

    ny, nx = stack.shape[1], stack.shape[2]
    data_full = np.full((24, ny, nx), np.nan, np.float32)
    
    order = np.argsort(hours)
    for i in order:
        t = hours[i]
        idx = int((t - start).total_seconds() // 3600)
        if 0 <= idx < 24:
            data_full[idx] = stack[i]

    if full_time.tz is not None:
        full_time_utc = full_time.tz_convert('UTC').tz_localize(None)
        time_attrs = {
            "standard_name": "time",
            "long_name": "time", 
            "timezone": "UTC (converted from KST)"
        }
    else:
        full_time_utc = full_time
        time_attrs = {"standard_name": "time", "long_name": "time"}

    ds = xr.Dataset(
        data_vars={
            "precipitation": (("time", "y", "x"), data_full,
                              {"units": "kg m-2 s-1", "long_name": "LDAPS precipitation rate"})
        },
        coords={
            "time": (("time",), full_time_utc, time_attrs),
            "y": np.arange(ny),
            "x": np.arange(nx),
            "latitude": (("y", "x"), lat2d, {"standard_name": "latitude", "units": "degrees_north"}),
            "longitude": (("y", "x"), lon2d, {"standard_name": "longitude", "units": "degrees_east"}),
        },
        attrs={
            "source": "KMA LDAPS",
            "Conventions": "CF-1.8"
        }
    )

    comp = dict(zlib=True, complevel=4)
    encoding = {
        "precipitation": dict(chunksizes=(1, min(256, ny), min(256, nx)), **comp, dtype="float32"),
        "latitude": dict(chunksizes=(min(256, ny), min(256, nx)), **comp, dtype="float32"),
        "longitude": dict(chunksizes=(min(256, ny), min(256, nx)), **comp, dtype="float32"),
    }

    ds.to_netcdf(out_path, encoding=encoding)
    print(f"[SAVE] {os.path.basename(out_path)}")
    ds.close()


def select_forecast_times_for_day(day):
    """하루 24시간을 커버하는 예보시간 조합 생성"""
    kst = day
    def bt(d, h): return d.strftime("%Y%m%d") + f"{h:02d}00"
    
    calls = []
    for init_hour in [0, 6, 12, 18]:
        for lead in range(0, 6):
            calls.append((bt(kst, init_hour), lead))
    
    return calls


def create_raw_wgs84_tif_rasterio(data_array, lats, lons, output_tif):
    """
    rasterio를 사용하여 2D 위경도 배열로부터 WGS84 GeoTIFF 생성
    GDAL HDF5 의존성 문제 해결
    """
    # 데이터 배열이 xarray인 경우 numpy로 변환
    if hasattr(data_array, 'values'):
        data = data_array.values.astype(np.float32)
    else:
        data = data_array.astype(np.float32)
    
    # 경계 계산 (위경도 2D 배열)
    lon_min, lon_max = float(lons.min()), float(lons.max())
    lat_min, lat_max = float(lats.min()), float(lats.max())
    
    height, width = data.shape
    
    # Affine transform 생성
    transform = from_bounds(lon_min, lat_min, lon_max, lat_max, width, height)
    
    # GeoTIFF 저장
    with rasterio.open(
        output_tif,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=rasterio.float32,
        crs='EPSG:4326',
        transform=transform,
        compress='deflate',
        predictor=2,
        tiled=True,
        nodata=np.nan
    ) as dst:
        dst.write(data, 1)
    
    return output_tif


def reproject_to_5179_rasterio(input_wgs84_tif, output_5179_tif, target_res=30):
    """
    rasterio를 사용하여 WGS84 → EPSG:5179, 30m 해상도 재투영
    최적화: COG(Cloud Optimized GeoTIFF) 포맷 사용
    """
    with rasterio.open(input_wgs84_tif) as src:
        # 목표 좌표계와 해상도 설정
        transform, width, height = calculate_default_transform(
            src.crs,
            TARGET_CRS,
            src.width,
            src.height,
            *src.bounds,
            resolution=(target_res, target_res)
        )
        
        # COG(Cloud Optimized GeoTIFF) 최적화 설정
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': TARGET_CRS,
            'transform': transform,
            'width': width,
            'height': height,
            'driver': 'GTiff',
            'dtype': rasterio.float32,
            'compress': 'deflate',
            'predictor': 2,
            'tiled': True,
            'blockxsize': 512,
            'blockysize': 512,
            'BIGTIFF': 'IF_SAFER',  # 4GB 초과 시 BigTIFF 자동 전환
            'NUM_THREADS': 'ALL_CPUS'  # 멀티스레드 압축
        })
        
        with rasterio.open(output_5179_tif, 'w', **kwargs) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=TARGET_CRS,
                resampling=Resampling.cubic,
                num_threads=4  # 재투영 멀티스레딩
            )
    
    return output_5179_tif


print("\n함수 정의 완료!")
print("\n워크플로우:")
print("  1단계: GRIB 다운로드 → NetCDF 변환 → 7일/3일 누적, 1시간 최대 계산")
print("  2단계: rasterio로 WGS84 TIF 생성 (GDAL HDF5 의존성 문제 해결)")
print("  3단계: rasterio로 EPSG:5179 30m TIF 생성 (Cubic 보간)")
print("\n다음 셀을 실행하여 Step 1 처리를 시작하세요.")


Step 1: LDAPS 강우 지표 생성
처리 기간: 2019-08-17 ~ 2020-02-28
원본 저장 (raw):
  - GRIB: D:\Landslide\data\raw\LDAPS\GRIB
  - NetCDF: D:\Landslide\data\raw\LDAPS\NetCDF
  - TIF (WGS84): D:\Landslide\data\raw\LDAPS\TIF_WGS84
처리 저장 (processed):
  - TIF (5179, 30m): D:\Landslide\data\processed\gyeongnam\LDAPS\TIF_5179_30m_20190817_20200228_original
좌표계: WGS84 → EPSG:5179
보간법: Cubic (rasterio)
해상도: ~1.5km → 30m

함수 정의 완료!

워크플로우:
  1단계: GRIB 다운로드 → NetCDF 변환 → 7일/3일 누적, 1시간 최대 계산
  2단계: rasterio로 WGS84 TIF 생성 (GDAL HDF5 의존성 문제 해결)
  3단계: rasterio로 EPSG:5179 30m TIF 생성 (Cubic 보간)

다음 셀을 실행하여 Step 1 처리를 시작하세요.


In [3]:
# ============================================================
# Step 1 실행 - LDAPS 데이터 다운로드 및 처리
# ============================================================

print("\nStep 1 처리 시작...")

# 🔧 다운로드 파라미터 설정
DOWNLOAD_RETRY_COUNT = 3          # 최대 재시도 횟수
DOWNLOAD_RETRY_DELAY = 5          # 재시도 간 대기 시간(초) - Exponential backoff 적용됨

print(f"\n⚙️  다운로드 설정:")
print(f"  - 최대 재시도: {DOWNLOAD_RETRY_COUNT}회")
print(f"  - 재시도 대기: {DOWNLOAD_RETRY_DELAY}초 (1차), {DOWNLOAD_RETRY_DELAY*2}초 (2차), {DOWNLOAD_RETRY_DELAY*3}초 (3차)")
print(f"  - 타임아웃: 연결 30초, 읽기 300초")
print(f"  - 재시도 정책: Exponential Backoff")
print("="*70)

date_range = pd.date_range(START_DATE, END_DATE, freq="D")
print(f"\n총 {len(date_range)}일 처리 예정\n")

rolling_7d = deque(maxlen=7)
rolling_3d = deque(maxlen=3)
daily_sums = {}

processed_count = 0
error_count = 0
skipped_count = 0
download_errors = 0
download_success = 0

for day_idx, day in enumerate(date_range):
    day_str = day.strftime('%Y%m%d')
    
    # ============================================================
    # 이미 생성된 파일 체크 (3개 파일 모두 있으면 스킵)
    # ============================================================
    out_acc7 = os.path.join(TIF_5179_DIR, "acc7d", f"{day_str}_acc7d_mm_5179_30m.tif")
    out_acc3 = os.path.join(TIF_5179_DIR, "acc3d", f"{day_str}_acc3d_mm_5179_30m.tif")
    out_peak = os.path.join(TIF_5179_DIR, "peak1h", f"{day_str}_peak1h_mm_5179_30m.tif")
    
    all_exist = all([
        os.path.exists(out_acc7) and os.path.getsize(out_acc7) > 0,
        os.path.exists(out_acc3) and os.path.getsize(out_acc3) > 0,
        os.path.exists(out_peak) and os.path.getsize(out_peak) > 0
    ])
    
    if all_exist:
        print(f"[{day_idx+1}/{len(date_range)}] {day.date()} - [SKIP] 이미 처리됨")
        skipped_count += 1
        
        # rolling 계산을 위해 NetCDF 로드 (rolling window 유지)
        output_nc = os.path.join(NETCDF_DIR, f"LDAPS_Rain_{day_str}.nc")
        if os.path.exists(output_nc):
            try:
                with xr.open_dataset(output_nc) as ds:
                    precip_data = ds['precipitation']
                    if precip_data.attrs.get('units') == 'kg m-2 s-1':
                        precip_data = precip_data * 3600.0
                    if "time" in precip_data.dims:
                        sum_day = precip_data.sum(dim="time", skipna=True)
                    else:
                        sum_day = precip_data
                    daily_sums[day_str] = sum_day
                    rolling_7d.append(sum_day)
                    rolling_3d.append(sum_day)
            except Exception as e:
                print(f"    [WARNING] NetCDF 로드 실패 (rolling 계산 불가): {e}")
        
        continue
    
    print(f"\n{'='*70}")
    print(f"[{day_idx+1}/{len(date_range)}] {day.date()}")
    print(f"{'='*70}")
    
    try:
        # 1) 예보시간 조합 생성
        forecast_times = select_forecast_times_for_day(day.to_pydatetime())
        
        daily_data = []
        valid_times = []
        reference_lat2d = None
        reference_lon2d = None
        
        # 2) GRIB 다운로드 및 데이터 수집
        for base_time, lead_hour in forecast_times:
            for variable in VARIABLES:
                # ✨ 개선: 재시도 파라미터 명시적 전달
                grib_file = download_grib_file(
                    SERVICE_KEY, 
                    base_time, 
                    lead_hour, 
                    variable, 
                    GRIB_DIR,
                    max_retries=DOWNLOAD_RETRY_COUNT,
                    retry_delay=DOWNLOAD_RETRY_DELAY
                )
                
                if grib_file is None:
                    download_errors += 1
                    continue
                
                download_success += 1
                
                try:
                    ds = grib_to_xarray(grib_file)
                    if ds is None:
                        continue
                    
                    if reference_lat2d is None:
                        try:
                            reference_lat2d, reference_lon2d = pick_reference_latlon_from_ds(ds)
                            print(f"[INFO] 좌표 추출: {reference_lat2d.shape}")
                        except Exception as e:
                            print(f"[WARNING] 좌표 추출 실패: {e}")
                    
                    data_vars = list(ds.data_vars.keys())
                    if not data_vars:
                        ds.close()
                        continue
                    
                    data_var = data_vars[0]
                    data_array = ds[data_var].values
                    
                    base_dt = datetime.strptime(base_time, "%Y%m%d%H%M")
                    if USE_KST:
                        base_dt = base_dt.replace(tzinfo=KST)
                    valid_time = base_dt + timedelta(hours=lead_hour)
                    
                    if valid_time.date() == day.date():
                        daily_data.append(data_array)
                        valid_times.append(valid_time)
                    
                    ds.close()
                    
                except Exception as e:
                    print(f"[ERROR] GRIB 처리 실패: {e}")
                    continue
                
                time.sleep(0.1)
        
        if not daily_data:
            print(f"[SKIP] 데이터 없음: {day.date()}")
            error_count += 1
            continue
            
        if reference_lat2d is None:
            print(f"[ERROR] 좌표 없음: {day.date()}")
            error_count += 1
            continue
        
        # 3) NetCDF 생성
        data_stack = np.stack(daily_data, axis=0)
        print(f"[INFO] 수집 완료: {len(daily_data)}개 시간대")
        
        output_nc = os.path.join(NETCDF_DIR, f"LDAPS_Rain_{day_str}.nc")
        build_daily_netcdf(day, data_stack, valid_times, reference_lat2d, reference_lon2d, output_nc)
        
        # 4) NetCDF 로드 및 통계 계산
        with xr.open_dataset(output_nc) as ds:
            precip_data = ds['precipitation']
            
            if precip_data.attrs.get('units') == 'kg m-2 s-1':
                precip_data = precip_data * 3600.0
                precip_data.attrs['units'] = 'mm/h'
            
            lats = ds['latitude'].values
            lons = ds['longitude'].values
            
            if "time" in precip_data.dims:
                sum_day = precip_data.sum(dim="time", skipna=True)
                sum_day.attrs['long_name'] = 'Daily accumulated precipitation'
                sum_day.attrs['units'] = 'mm'
                
                peak_day = precip_data.max(dim="time", skipna=True) 
                peak_day.attrs['long_name'] = 'Daily peak precipitation rate'
                peak_day.attrs['units'] = 'mm/h'
            else:
                sum_day = precip_data
                peak_day = precip_data
            
            daily_sums[day_str] = sum_day
            rolling_7d.append(sum_day)
            rolling_3d.append(sum_day)
            
            # 5) 누적 강우 계산
            if len(rolling_7d) > 1:
                acc7 = xr.concat(list(rolling_7d), dim="rolling_day").sum(dim="rolling_day", skipna=True)
                acc7.attrs['long_name'] = '7-day accumulated precipitation'
                acc7.attrs['units'] = 'mm'
            else:
                acc7 = sum_day
            
            if len(rolling_3d) > 1:
                acc3 = xr.concat(list(rolling_3d), dim="rolling_day").sum(dim="rolling_day", skipna=True)
                acc3.attrs['long_name'] = '3-day accumulated precipitation'
                acc3.attrs['units'] = 'mm'
            else:
                acc3 = sum_day
            
            # 6) GeoTIFF 생성 (rasterio 사용)
            try:
                # Step 1: WGS84 TIF 생성 (rasterio)
                raw_acc7 = os.path.join(TIF_RAW_DIR, "acc7d", f"{day_str}_acc7d_mm_wgs84.tif")
                raw_acc3 = os.path.join(TIF_RAW_DIR, "acc3d", f"{day_str}_acc3d_mm_wgs84.tif")
                raw_peak = os.path.join(TIF_RAW_DIR, "peak1h", f"{day_str}_peak1h_mm_wgs84.tif")
                
                print(f"    [SAVE] WGS84 TIF 생성 (rasterio)...")
                create_raw_wgs84_tif_rasterio(acc7, lats, lons, raw_acc7)
                create_raw_wgs84_tif_rasterio(acc3, lats, lons, raw_acc3)
                create_raw_wgs84_tif_rasterio(peak_day, lats, lons, raw_peak)
                
                # Step 2: EPSG:5179 30m TIF 생성 (rasterio)
                print(f"    [INTERP] Cubic 보간 (1.5km → 30m, rasterio)...")
                reproject_to_5179_rasterio(raw_acc7, out_acc7, target_res=TARGET_RES)
                reproject_to_5179_rasterio(raw_acc3, out_acc3, target_res=TARGET_RES)
                reproject_to_5179_rasterio(raw_peak, out_peak, target_res=TARGET_RES)
                
                # 통계 출력
                acc7_max = float(acc7.max().values) if not np.isnan(acc7.max().values) else 0.0
                acc3_max = float(acc3.max().values) if not np.isnan(acc3.max().values) else 0.0
                peak_max = float(peak_day.max().values) if not np.isnan(peak_day.max().values) else 0.0
                
                print(f"    [RESULT] 7일: {acc7_max:.1f}mm | 3일: {acc3_max:.1f}mm | 1h최대: {peak_max:.1f}mm/h")
                
                processed_count += 1
                
            except Exception as e:
                print(f"    [ERROR] GeoTIFF 생성 실패: {e}")
                import traceback
                traceback.print_exc()
                error_count += 1
    
    except Exception as e:
        print(f"[ERROR] 처리 실패 {day.date()}: {e}")
        import traceback
        traceback.print_exc()
        error_count += 1
        continue

print(f"\n{'='*70}")
print("Step 1 완료!")
print(f"{'='*70}")
print(f"신규 처리: {processed_count}일")
print(f"스킵 (이미 존재): {skipped_count}일")
print(f"실패: {error_count}일")
print(f"다운로드 성공: {download_success}개 파일")
print(f"다운로드 실패: {download_errors}개 파일")
print(f"전체: {processed_count + skipped_count + error_count}/{len(date_range)}일")
print(f"\n출력 디렉토리:")
print(f"  - 원본 1.5km WGS84: {TIF_RAW_DIR}")
print(f"  - 처리 30m 5179: {TIF_5179_DIR}")
print(f"{'='*70}")


Step 1 처리 시작...

⚙️  다운로드 설정:
  - 최대 재시도: 3회
  - 재시도 대기: 5초 (1차), 10초 (2차), 15초 (3차)
  - 타임아웃: 연결 30초, 읽기 300초
  - 재시도 정책: Exponential Backoff

총 196일 처리 예정


[1/196] 2019-08-17
[DOWNLOAD] l015_201908170000_lspr_h000.gb2
[SUCCESS] Downloaded l015_201908170000_lspr_h000.gb2 (995402 bytes)
[INFO] 좌표 추출: (781, 602)
[DOWNLOAD] l015_201908170000_lspr_h001.gb2
[SUCCESS] Downloaded l015_201908170000_lspr_h001.gb2 (995402 bytes)
[DOWNLOAD] l015_201908170000_lspr_h002.gb2
[SUCCESS] Downloaded l015_201908170000_lspr_h002.gb2 (995402 bytes)
[DOWNLOAD] l015_201908170000_lspr_h003.gb2
[SUCCESS] Downloaded l015_201908170000_lspr_h003.gb2 (995402 bytes)
[DOWNLOAD] l015_201908170000_lspr_h004.gb2
[SUCCESS] Downloaded l015_201908170000_lspr_h004.gb2 (995402 bytes)
[DOWNLOAD] l015_201908170000_lspr_h005.gb2
[SUCCESS] Downloaded l015_201908170000_lspr_h005.gb2 (995402 bytes)
[DOWNLOAD] l015_201908170600_lspr_h000.gb2
[SUCCESS] Downloaded l015_201908170600_lspr_h000.gb2 (995402 bytes)
[DOWNLOAD] l015_201

In [4]:
# ============================================================
# Step 2A - 경남 영역 클립 (한반도 전체 → 경남 바운딩박스)
# ============================================================

from rasterio.mask import mask as rasterio_mask
from shapely.geometry import box

print("\n" + "="*70)
print("Step 2A: 경남 영역 클립 (바운딩박스 기반)")
print("="*70)

# 경로 설정
RAW_LDAPS_DIR = r"D:\Landslide\data\processed\gyeongnam\LDAPS\TIF_5179_30m_20190817_20200228_original"
PROCESSED_BASE_DIR = r'D:\Landslide\data\processed\gyeongnam\LDAPS'
CLIPPED_DIR = os.path.join(PROCESSED_BASE_DIR, f'TIF_5179_30m_20190817_20200228_clipped')
SLOPE_UNITS_PATH = r"D:\Landslide\data\processed\gyeongnam\SU\slope_properties_v3.gpkg"

os.makedirs(os.path.join(CLIPPED_DIR, "acc7d"), exist_ok=True)
os.makedirs(os.path.join(CLIPPED_DIR, "acc3d"), exist_ok=True)
os.makedirs(os.path.join(CLIPPED_DIR, "peak1h"), exist_ok=True)

print(f"원본 LDAPS TIF: {RAW_LDAPS_DIR}")
print(f"클립된 TIF 저장: {CLIPPED_DIR}")
print(f"사면 단위 파일: {SLOPE_UNITS_PATH}")
print("="*70)


def get_bbox_from_slope_units(slope_units_path, buffer_m=1000):
    """
    사면 단위 GeoPackage에서 바운딩박스 추출 (버퍼 포함)
    
    Args:
        slope_units_path: 사면 단위 GPKG 경로
        buffer_m: 버퍼 거리 (미터)
    
    Returns:
        GeoDataFrame with bounding box polygon
    """
    print("\n[LOAD] 사면 단위 바운딩박스 계산 중...")
    
    # 사면 단위 로드 (빠른 바운딩박스 계산만)
    slope_units = gpd.read_file(slope_units_path)
    print(f"  사면 개수: {len(slope_units):,}개")
    print(f"  좌표계: {slope_units.crs}")
    
    if slope_units.crs.to_string() != "EPSG:5179":
        print(f"  좌표계 변환: {slope_units.crs} → EPSG:5179")
        slope_units = slope_units.to_crs("EPSG:5179")
    
    # Total bounds 추출 (최소/최대 좌표)
    minx, miny, maxx, maxy = slope_units.total_bounds
    print(f"  원본 범위: ({minx:.1f}, {miny:.1f}) ~ ({maxx:.1f}, {maxy:.1f})")
    
    # 버퍼 추가
    minx -= buffer_m
    miny -= buffer_m
    maxx += buffer_m
    maxy += buffer_m
    
    print(f"  버퍼 {buffer_m}m 적용 후: ({minx:.1f}, {miny:.1f}) ~ ({maxx:.1f}, {maxy:.1f})")
    
    # 바운딩박스 폴리곤 생성
    bbox_poly = box(minx, miny, maxx, maxy)
    bbox_gdf = gpd.GeoDataFrame({'geometry': [bbox_poly]}, crs="EPSG:5179")
    
    # 크기 계산
    width_km = (maxx - minx) / 1000
    height_km = (maxy - miny) / 1000
    print(f"  바운딩박스 크기: {width_km:.1f}km × {height_km:.1f}km")
    
    return bbox_gdf


def clip_raster_to_bbox(input_tif, output_tif, bbox_gdf):
    """
    한반도 전체 래스터를 바운딩박스로 클립 (고속)
    
    Args:
        input_tif: 원본 TIF 경로
        output_tif: 클립된 TIF 저장 경로
        bbox_gdf: 바운딩박스 GeoDataFrame
    """
    with rasterio.open(input_tif) as src:
        # 바운딩박스를 래스터 좌표계로 변환 (필요시)
        if bbox_gdf.crs != src.crs:
            bbox_gdf = bbox_gdf.to_crs(src.crs)
        
        # 클립 수행 (단순 바운딩박스이므로 매우 빠름)
        out_image, out_transform = rasterio_mask(
            src, 
            bbox_gdf.geometry,
            crop=True,
            all_touched=False,
            nodata=np.nan
        )
        
        # 메타데이터 업데이트
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "compress": "deflate",
            "predictor": 2,
            "tiled": True,
            "blockxsize": 512,
            "blockysize": 512
        })
        
        # 클립된 래스터 저장
        with rasterio.open(output_tif, "w", **out_meta) as dest:
            dest.write(out_image)
    
    return out_image.shape[1:]  # (height, width)


print("\n함수 정의 완료!")
print("\n워크플로우:")
print("  1. 사면 단위 total_bounds 추출 (1km 버퍼)")
print("  2. 단순 사각형 바운딩박스 생성")
print("  3. 바운딩박스로 한반도 전체 TIF 클립")

print("  - 클립 속도: 폴리곤 대비 10배 이상 향상")
print("  - 메모리: 한반도 전체 대비 90% 절감")
print("\n다음 셀에서 클립 작업을 실행하세요.")


Step 2A: 경남 영역 클립 (바운딩박스 기반)
원본 LDAPS TIF: D:\Landslide\data\processed\gyeongnam\LDAPS\TIF_5179_30m_20190817_20200228_original
클립된 TIF 저장: D:\Landslide\data\processed\gyeongnam\LDAPS\TIF_5179_30m_20190817_20200228_clipped
사면 단위 파일: D:\Landslide\data\processed\gyeongnam\SU\slope_properties_v3.gpkg

함수 정의 완료!

워크플로우:
  1. 사면 단위 total_bounds 추출 (1km 버퍼)
  2. 단순 사각형 바운딩박스 생성
  3. 바운딩박스로 한반도 전체 TIF 클립
  - 클립 속도: 폴리곤 대비 10배 이상 향상
  - 메모리: 한반도 전체 대비 90% 절감

다음 셀에서 클립 작업을 실행하세요.


In [5]:
# ============================================================
# Step 2A 실행 - 바운딩박스 생성 및 저장
# ============================================================

print("\nStep 2A-1: 바운딩박스 생성 및 저장...\n")

# 1. 바운딩박스 생성
gyeongnam_bbox = get_bbox_from_slope_units(SLOPE_UNITS_PATH, buffer_m=1000)

# 2. 바운딩박스 저장 (점검용)
bbox_output = os.path.join(PROCESSED_BASE_DIR, "gyeongnam_bbox_1km_buffer.gpkg")
gyeongnam_bbox.to_file(bbox_output, driver="GPKG")

print(f"\n[SAVE] 바운딩박스 저장: {bbox_output}")
print(f"  좌표계: {gyeongnam_bbox.crs}")
print(f"  영역: {gyeongnam_bbox.total_bounds}")
print(f"\n💡 QGIS에서 바운딩박스를 확인한 후 다음 셀을 실행하세요:")
print(f"   1. 바운딩박스: {bbox_output}")
print(f"   2. 사면 단위: {SLOPE_UNITS_PATH}")
print(f"   3. 한반도 전체 래스터 샘플: {RAW_LDAPS_DIR}/acc7d/20200301_acc7d_mm_5179_30m.tif")
print("="*70)


Step 2A-1: 바운딩박스 생성 및 저장...


[LOAD] 사면 단위 바운딩박스 계산 중...
  사면 개수: 89,529개
  좌표계: EPSG:5179
  원본 범위: (1006911.5, 1616185.1) ~ (1156101.5, 1768225.1)
  버퍼 1000m 적용 후: (1005911.5, 1615185.1) ~ (1157101.5, 1769225.1)
  바운딩박스 크기: 151.2km × 154.0km

[SAVE] 바운딩박스 저장: D:\Landslide\data\processed\gyeongnam\LDAPS\gyeongnam_bbox_1km_buffer.gpkg
  좌표계: EPSG:5179
  영역: [1005911.53850933 1615185.12261852 1157101.53850933 1769225.12261852]

💡 QGIS에서 바운딩박스를 확인한 후 다음 셀을 실행하세요:
   1. 바운딩박스: D:\Landslide\data\processed\gyeongnam\LDAPS\gyeongnam_bbox_1km_buffer.gpkg
   2. 사면 단위: D:\Landslide\data\processed\gyeongnam\SU\slope_properties_v3.gpkg
   3. 한반도 전체 래스터 샘플: D:\Landslide\data\processed\gyeongnam\LDAPS\TIF_5179_30m_20190817_20200228_original/acc7d/20200301_acc7d_mm_5179_30m.tif


In [6]:
# ============================================================
# Step 2A-2 실행 - 바운딩박스 클립 수행
# ============================================================

print("\nStep 2A-2: 바운딩박스 클립 수행...\n")

# 바운딩박스 로드 (이전 셀에서 생성한 파일)
bbox_path = os.path.join(PROCESSED_BASE_DIR, "gyeongnam_bbox_1km_buffer.gpkg")

if not os.path.exists(bbox_path):
    print(f"[ERROR] 바운딩박스 파일이 없습니다: {bbox_path}")
    print("이전 셀(Step 2A-1)을 먼저 실행하세요.")
else:
    print(f"[LOAD] 바운딩박스: {bbox_path}")
    gyeongnam_bbox = gpd.read_file(bbox_path)
    print(f"  좌표계: {gyeongnam_bbox.crs}")
    print(f"  영역: {gyeongnam_bbox.total_bounds}")
    
    # 클립할 파일 목록 생성
    variable_types = ['acc7d', 'acc3d', 'peak1h']
    total_clipped = 0
    total_skipped = 0
    total_failed = 0
    
    for var_type in variable_types:
        print(f"\n{'='*70}")
        print(f"변수: {var_type}")
        print(f"{'='*70}")
        
        input_dir = os.path.join(RAW_LDAPS_DIR, var_type)
        output_dir = os.path.join(CLIPPED_DIR, var_type)
        
        if not os.path.exists(input_dir):
            print(f"[ERROR] 원본 디렉토리가 없습니다: {input_dir}")
            continue
        
        tif_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.tif')])
        print(f"총 {len(tif_files)}개 파일 처리 예정")
        
        for idx, filename in enumerate(tif_files):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            
            # 이미 처리된 파일은 스킵
            if os.path.exists(output_path):
                if (idx + 1) % 50 == 0:
                    print(f"  [{idx+1}/{len(tif_files)}] {filename} - [SKIP]")
                total_skipped += 1
                continue
            
            try:
                # 바운딩박스로 클립 (고속)
                clipped_shape = clip_raster_to_bbox(input_path, output_path, gyeongnam_bbox)
                
                # 간헐적 진행 상황 출력
                if (idx + 1) % 10 == 0 or idx == 0:
                    file_size = os.path.getsize(output_path) / 1024**2
                    print(f"  [{idx+1}/{len(tif_files)}] {filename} → {clipped_shape} ({file_size:.1f} MB)")
                
                total_clipped += 1
                
            except Exception as e:
                print(f"  [ERROR] {filename} 클립 실패: {e}")
                import traceback
                traceback.print_exc()
                total_failed += 1
                continue
    
    print(f"\n{'='*70}")
    print("Step 2A-2 클립 완료!")
    print(f"{'='*70}")
    print(f"신규 클립: {total_clipped}개")
    print(f"스킵 (이미 존재): {total_skipped}개")
    print(f"실패: {total_failed}개")
    print(f"전체: {total_clipped + total_skipped + total_failed}개")
    print(f"\n출력 디렉토리: {CLIPPED_DIR}")
    print("💡 클립 완료 → Step 2B (통계 추출) 실행 가능")
    print("="*70)


Step 2A-2: 바운딩박스 클립 수행...

[LOAD] 바운딩박스: D:\Landslide\data\processed\gyeongnam\LDAPS\gyeongnam_bbox_1km_buffer.gpkg
  좌표계: EPSG:5179
  영역: [1005911.53850933 1615185.12261852 1157101.53850933 1769225.12261852]

변수: acc7d
총 196개 파일 처리 예정
  [1/196] 20190817_acc7d_mm_5179_30m.tif → (5136, 5041) (11.3 MB)
  [10/196] 20190826_acc7d_mm_5179_30m.tif → (5136, 5041) (68.6 MB)
  [20/196] 20190905_acc7d_mm_5179_30m.tif → (5136, 5041) (58.7 MB)
  [30/196] 20190915_acc7d_mm_5179_30m.tif → (5136, 5041) (59.2 MB)
  [40/196] 20190925_acc7d_mm_5179_30m.tif → (5136, 5041) (50.1 MB)
  [50/196] 20191005_acc7d_mm_5179_30m.tif → (5136, 5041) (50.9 MB)
  [60/196] 20191015_acc7d_mm_5179_30m.tif → (5136, 5041) (63.5 MB)
  [70/196] 20191025_acc7d_mm_5179_30m.tif → (5136, 5041) (69.5 MB)
  [80/196] 20191104_acc7d_mm_5179_30m.tif → (5136, 5041) (34.8 MB)
  [90/196] 20191114_acc7d_mm_5179_30m.tif → (5136, 5041) (60.4 MB)
  [100/196] 20191124_acc7d_mm_5179_30m.tif → (5136, 5041) (56.8 MB)
  [110/196] 20191204_acc7d

In [7]:
# ============================================================
# Step 2B - 사면별 통계 추출 (최적화: NumPy 벡터화 + 증분 업데이트)
# ============================================================

from rasterio.features import rasterize
from collections import defaultdict
import scipy.ndimage as ndimage
import glob

print("\n" + "="*70)
print("Step 2B: 사면별 통계 추출 (최적화: 증분 업데이트)")
print("="*70)

# 경로 설정
PROCESSED_BASE_DIR = r'D:\Landslide\data\processed\gyeongnam\LDAPS'
SLOPE_UNITS_PATH = r"D:\Landslide\data\processed\gyeongnam\SU\slope_properties_v3.gpkg"
ZONE_RASTER_PATH = os.path.join(PROCESSED_BASE_DIR, "zone_raster_gyeongnam_5179_30m.tif")

# 📁 CSV 파일 경로 (명확한 네이밍)
RAW_LONG_CSV = os.path.join(PROCESSED_BASE_DIR, "statistics", "ldaps_statistics_long_format.csv")  # 증분 업데이트용
WIDE_PIVOT_CSV = os.path.join(PROCESSED_BASE_DIR, "statistics", "ldaps_statistics_wide_format.csv")  # 모델 학습용

# 통계 디렉토리 생성
os.makedirs(os.path.join(PROCESSED_BASE_DIR, "statistics"), exist_ok=True)

print(f"Zone Raster: {ZONE_RASTER_PATH}")
print(f"\n📁 출력 CSV 파일:")
print(f"  ✅ Long Format (증분 업데이트): {os.path.basename(RAW_LONG_CSV)}")
print(f"     → 각 행 = (사면, 날짜, 변수, 통계값)")
print(f"  ✅ Wide Format (모델 학습용):  {os.path.basename(WIDE_PIVOT_CSV)}")
print(f"     → 각 열 = 날짜별 변수 (acc7d_mean_20190101, ...)")
print("="*70)


def find_clipped_folders(base_dir):
    """
    클립된 LDAPS 폴더 목록을 찾아서 반환
    
    패턴: TIF_5179_30m_YYYYMMDD_YYYYMMDD_clipped 또는 TIF_5179_30m_YYYYMMDD_YYYYMMDD
    
    Returns:
        List of (folder_path, start_date, end_date)
    """
    print("\n[SCAN] 클립된 LDAPS 폴더 검색 중...")
    print(f"  검색 경로: {base_dir}")
    
    folders = []
    
    # base_dir 내의 모든 디렉토리 검색
    try:
        all_items = os.listdir(base_dir)
    except Exception as e:
        print(f"  [ERROR] 디렉토리 읽기 실패: {e}")
        return folders
    
    for item in all_items:
        item_path = os.path.join(base_dir, item)
        
        # 디렉토리만 확인
        if not os.path.isdir(item_path):
            continue
        
        # TIF_5179_30m로 시작하는 폴더만 (statistics, zone_raster 등 제외)
        if not item.startswith("TIF_5179_30m_"):
            continue
        
        # "_clipped" 제거 후 파싱
        folder_name_clean = item.replace("_clipped", "")
        parts = folder_name_clean.split('_')
        
        # TIF_5179_30m_YYYYMMDD_YYYYMMDD 형식 검증
        # parts[0] = "TIF", parts[1] = "5179", parts[2] = "30m", parts[3] = 시작날짜, parts[4] = 종료날짜
        if len(parts) >= 5:
            try:
                start_date = parts[3]
                end_date = parts[4]
                
                # 날짜 형식 검증 (YYYYMMDD - 8자리 숫자)
                if len(start_date) == 8 and len(end_date) == 8 and start_date.isdigit() and end_date.isdigit():
                    folders.append((item_path, start_date, end_date))
                    print(f"  ✅ 발견: {item}")
                    print(f"      → 기간: {start_date} ~ {end_date}")
            except Exception as e:
                print(f"  ⚠️  스킵: {item} (파싱 실패: {e})")
                continue
    
    # 시작 날짜 기준 정렬
    folders.sort(key=lambda x: x[1])
    
    print(f"\n총 {len(folders)}개 폴더 발견")
    
    if len(folders) == 0:
        print("\n💡 문제 해결 방법:")
        print("  1. 폴더명이 'TIF_5179_30m_YYYYMMDD_YYYYMMDD_clipped' 형식인지 확인")
        print("  2. Step 2A에서 클립 작업이 완료되었는지 확인")
        print(f"  3. 현재 경로 확인: {base_dir}")
    
    return folders


def load_existing_statistics(long_csv_path):
    """
    기존 통계 CSV 로드 (Long Format)
    
    Args:
        long_csv_path: Long format CSV 경로
    
    Returns:
        DataFrame or None, set of (date, variable) tuples
    """
    if os.path.exists(long_csv_path):
        print(f"\n[LOAD] 기존 통계 파일 로드: {os.path.basename(long_csv_path)}")
        existing_df = pd.read_csv(long_csv_path)
        print(f"  기존 레코드: {len(existing_df):,}개")
        
        # 이미 계산된 (날짜, 변수) 조합 추출
        computed_keys = set(
            zip(existing_df['date'].astype(str), existing_df['variable'])
        )
        
        unique_dates = existing_df['date'].nunique()
        print(f"  고유 날짜: {unique_dates}개")
        print(f"  고유 (날짜, 변수) 조합: {len(computed_keys):,}개")
        
        return existing_df, computed_keys
    else:
        print(f"\n[INFO] 기존 통계 파일 없음 (새로 생성)")
        return None, set()


def create_zone_raster_from_clipped(slope_units, reference_tif, output_zone_path):
    """
    클립된 래스터를 기준으로 Zone Raster 생성
    
    Args:
        slope_units: 사면 단위 GeoDataFrame
        reference_tif: 클립된 TIF 파일 (크기/변환 정보 참조)
        output_zone_path: Zone Raster 저장 경로
    """
    print(f"\n[CREATE] Zone Raster 생성 중...")
    print(f"  기준 래스터: {os.path.basename(reference_tif)}")
    
    with rasterio.open(reference_tif) as ref_src:
        ref_shape = ref_src.shape
        ref_transform = ref_src.transform
        ref_crs = ref_src.crs
        
        print(f"  격자 크기: {ref_shape}")
        print(f"  좌표계: {ref_crs}")
    
    # 사면 폴리곤을 래스터화
    shapes = [(geom, cat_id) for geom, cat_id in zip(slope_units.geometry, slope_units['cat'])]
    
    print(f"  래스터화 대상: {len(shapes):,}개 폴리곤")
    
    zone_array = rasterize(
        shapes=shapes,
        out_shape=ref_shape,
        transform=ref_transform,
        fill=0,
        dtype=np.uint32,
        all_touched=False
    )
    
    # Zone Raster 저장
    with rasterio.open(
        output_zone_path,
        'w',
        driver='GTiff',
        height=ref_shape[0],
        width=ref_shape[1],
        count=1,
        dtype=np.uint32,
        crs=ref_crs,
        transform=ref_transform,
        compress='deflate',
        predictor=2,
        tiled=True,
        nodata=0
    ) as dst:
        dst.write(zone_array, 1)
    
    unique_zones = np.unique(zone_array[zone_array > 0])
    print(f"  [SAVE] Zone Raster 생성 완료")
    print(f"  고유 사면 ID: {len(unique_zones):,}개 (1 ~ {unique_zones.max()})")
    
    return zone_array, unique_zones


def compute_stats_vectorized(rainfall_tif, zone_array, zone_ids, date_str, variable_type):
    """
    NumPy 벡터화 기반 통계 계산 (scipy.ndimage 활용)
    
    최적화:
    - scipy.ndimage.labeled_comprehension 사용 (C 레벨 최적화)
    - zone별 반복 제거 (전체 zone 한번에 처리)
    - 메모리 효율적 통계 계산
    
    Args:
        rainfall_tif: 클립된 강우 TIF 경로
        zone_array: Zone Raster 배열
        zone_ids: 고유 zone ID 배열
        date_str: 날짜 문자열 (YYYYMMDD)
        variable_type: 변수 타입 (acc7d, acc3d, peak1h)
    
    Returns:
        Dictionary with statistics per zone
    """
    try:
        # 강우 데이터 로드
        with rasterio.open(rainfall_tif) as src:
            rainfall_data = src.read(1).astype(np.float32)
        
        # 크기 일치 확인
        if rainfall_data.shape != zone_array.shape:
            min_h = min(rainfall_data.shape[0], zone_array.shape[0])
            min_w = min(rainfall_data.shape[1], zone_array.shape[1])
            rainfall_data = rainfall_data[:min_h, :min_w]
            zone_array_used = zone_array[:min_h, :min_w]
        else:
            zone_array_used = zone_array
        
        # NaN을 0으로 변환 (통계 계산 단순화)
        rainfall_clean = np.nan_to_num(rainfall_data, nan=0.0)
        
        # 벡터화 통계 계산 (scipy.ndimage - C 레벨 최적화)
        means = ndimage.mean(rainfall_clean, labels=zone_array_used, index=zone_ids)
        maxs = ndimage.maximum(rainfall_clean, labels=zone_array_used, index=zone_ids)
        mins = ndimage.minimum(rainfall_clean, labels=zone_array_used, index=zone_ids)
        stds = ndimage.standard_deviation(rainfall_clean, labels=zone_array_used, index=zone_ids)
        
        # 픽셀 개수 계산
        counts = np.bincount(zone_array_used.ravel(), minlength=int(zone_ids.max())+1)[zone_ids]
        
        # 결과를 딕셔너리로 반환 (DataFrame 생성은 나중에 일괄 처리)
        results = {
            'date': date_str,
            'variable': variable_type,
            'zone_ids': zone_ids,
            'means': means,
            'maxs': maxs,
            'mins': mins,
            'stds': stds,
            'counts': counts
        }
        
        return results
    
    except Exception as e:
        print(f"    [ERROR] {date_str} 통계 계산 실패: {e}")
        return None


def save_combined_statistics(new_results_df, existing_df, long_csv_path, wide_csv_path):
    """
    신규 통계와 기존 통계를 병합하여 저장
    
    Args:
        new_results_df: 신규 계산된 통계 DataFrame
        existing_df: 기존 통계 DataFrame (None 가능)
        long_csv_path: Long Format CSV 저장 경로 (증분 업데이트용)
        wide_csv_path: Wide Format CSV 저장 경로 (모델 학습용)
    """
    print(f"\n[MERGE] 통계 병합 및 저장 중...")
    
    # 기존 데이터와 병합
    if existing_df is not None:
        combined_stats = pd.concat([existing_df, new_results_df], ignore_index=True)
        print(f"  기존: {len(existing_df):,}개 + 신규: {len(new_results_df):,}개 = 전체: {len(combined_stats):,}개")
    else:
        combined_stats = new_results_df
        print(f"  신규: {len(new_results_df):,}개")
    
    # 중복 제거 (혹시 모를 중복 방지)
    combined_stats = combined_stats.drop_duplicates(subset=['cat', 'date', 'variable'], keep='last')
    print(f"  중복 제거 후: {len(combined_stats):,}개")
    
    # ✅ Long Format 저장 (증분 업데이트용)
    combined_stats.to_csv(long_csv_path, index=False, encoding='utf-8-sig')
    print(f"\n  [SAVE] 📊 Long Format (증분 업데이트용)")
    print(f"         → {os.path.basename(long_csv_path)}")
    print(f"         → 각 행 = (사면ID, 날짜, 변수, 통계값)")
    
    # ✅ Wide Format으로 변환 (모델 학습용)
    print(f"\n  [PIVOT] Wide Format 변환 중...")
    
    variable_types = ['acc7d', 'acc3d', 'peak1h']
    pivot_tables = {}
    
    for var_type in variable_types:
        var_stats = combined_stats[combined_stats['variable'] == var_type]
        
        if len(var_stats) == 0:
            continue
        
        # mean pivot
        pivot_mean = var_stats.pivot(index='cat', columns='date', values='mean')
        pivot_mean.columns = [f'{var_type}_mean_{col}' for col in pivot_mean.columns]
        pivot_tables[f'{var_type}_mean'] = pivot_mean
        
        # max pivot
        pivot_max = var_stats.pivot(index='cat', columns='date', values='max')
        pivot_max.columns = [f'{var_type}_max_{col}' for col in pivot_max.columns]
        pivot_tables[f'{var_type}_max'] = pivot_max
    
    # Wide format DataFrame 생성
    final_df = pd.concat(pivot_tables.values(), axis=1)
    final_df.index.name = 'cat'
    final_df = final_df.reset_index()
    
    # Wide format 저장
    final_df.to_csv(wide_csv_path, index=False, encoding='utf-8-sig')
    
    print(f"\n  [SAVE] 📈 Wide Format (모델 학습용)")
    print(f"         → {os.path.basename(wide_csv_path)}")
    print(f"         → 각 열 = 날짜별 변수 (acc7d_mean_20190101, acc7d_max_20190101, ...)")
    print(f"         → 행: {len(final_df):,}개 사면")
    print(f"         → 열: {len(final_df.columns):,}개 (cat + 날짜별 통계)")


print("\n함수 정의 완료!")
print("\n💡 핵심 개선사항:")
print("  1. 여러 기간별 폴더 자동 검색 및 처리")
print("  2. 증분 업데이트: 이미 계산된 통계는 스킵")
print("  3. 기존 CSV에 신규 통계 추가 (병합)")
print("  4. scipy.ndimage 벡터 연산: zone별 반복 제거")
print("\n📁 출력 CSV 파일 형식:")
print("  ✅ Long Format: 각 행이 하나의 측정값 (증분 업데이트 쉬움)")
print("  ✅ Wide Format: 각 열이 날짜별 변수 (모델 학습에 바로 사용)")
print("\n예상 속도 향상: 기존 대비 5~10배 (신규 데이터만 처리)")
print("\n다음 셀에서 통계 추출을 실행하세요.")


Step 2B: 사면별 통계 추출 (최적화: 증분 업데이트)
Zone Raster: D:\Landslide\data\processed\gyeongnam\LDAPS\zone_raster_gyeongnam_5179_30m.tif

📁 출력 CSV 파일:
  ✅ Long Format (증분 업데이트): ldaps_statistics_long_format.csv
     → 각 행 = (사면, 날짜, 변수, 통계값)
  ✅ Wide Format (모델 학습용):  ldaps_statistics_wide_format.csv
     → 각 열 = 날짜별 변수 (acc7d_mean_20190101, ...)

함수 정의 완료!

💡 핵심 개선사항:
  1. 여러 기간별 폴더 자동 검색 및 처리
  2. 증분 업데이트: 이미 계산된 통계는 스킵
  3. 기존 CSV에 신규 통계 추가 (병합)
  4. scipy.ndimage 벡터 연산: zone별 반복 제거

📁 출력 CSV 파일 형식:
  ✅ Long Format: 각 행이 하나의 측정값 (증분 업데이트 쉬움)
  ✅ Wide Format: 각 열이 날짜별 변수 (모델 학습에 바로 사용)

예상 속도 향상: 기존 대비 5~10배 (신규 데이터만 처리)

다음 셀에서 통계 추출을 실행하세요.


In [8]:
# ============================================================
# Step 2B 실행 - 증분 통계 추출 (여러 기간 폴더 처리)
# ============================================================

import time

print("\nStep 2B 통계 추출 시작 (증분 업데이트)...\n")
start_time = time.time()

# 1. 클립된 LDAPS 폴더 검색
clipped_folders = find_clipped_folders(PROCESSED_BASE_DIR)

if not clipped_folders:
    print("[ERROR] 클립된 LDAPS 폴더를 찾을 수 없습니다!")
    print(f"경로 확인: {PROCESSED_BASE_DIR}")
else:
    # 2. 기존 통계 로드 (Long Format)
    existing_df, computed_keys = load_existing_statistics(RAW_LONG_CSV)
    
    # 3. 사면 단위 로드
    print(f"\n[LOAD] 사면 단위 로드 중...")
    slope_units = gpd.read_file(SLOPE_UNITS_PATH)
    print(f"  사면 개수: {len(slope_units):,}개")
    
    if slope_units.crs.to_string() != "EPSG:5179":
        slope_units = slope_units.to_crs("EPSG:5179")
    
    # 4. Zone Raster 생성 또는 로드
    print(f"\n[ZONE] Zone Raster 준비 중...")
    
    if not os.path.exists(ZONE_RASTER_PATH):
        # 첫 번째 폴더에서 참조 TIF 찾기
        first_folder = clipped_folders[0][0]
        acc7d_dir = os.path.join(first_folder, "acc7d")
        
        if os.path.exists(acc7d_dir):
            reference_tif = os.path.join(acc7d_dir, sorted(os.listdir(acc7d_dir))[0])
            zone_array, zone_ids = create_zone_raster_from_clipped(
                slope_units, reference_tif, ZONE_RASTER_PATH
            )
        else:
            print(f"[ERROR] acc7d 디렉토리를 찾을 수 없습니다: {acc7d_dir}")
            raise FileNotFoundError(f"acc7d directory not found")
    else:
        print(f"  기존 Zone Raster 사용: {os.path.basename(ZONE_RASTER_PATH)}")
        with rasterio.open(ZONE_RASTER_PATH) as src:
            zone_array = src.read(1)
            zone_ids = np.unique(zone_array[zone_array > 0])
        print(f"  고유 사면 ID: {len(zone_ids):,}개")
        print(f"  격자 크기: {zone_array.shape}")
        print(f"  메모리: {zone_array.nbytes / 1024**2:.1f} MB")
    
    # 5. 각 폴더별로 통계 계산
    print(f"\n{'='*70}")
    print("통계 계산 시작")
    print(f"{'='*70}")
    
    variable_types = ['acc7d', 'acc3d', 'peak1h']
    all_new_results = []  # 신규 계산된 결과만
    
    total_processed = 0
    total_skipped = 0
    
    for folder_idx, (folder_path, start_date, end_date) in enumerate(clipped_folders):
        folder_name = os.path.basename(folder_path)
        print(f"\n[{folder_idx+1}/{len(clipped_folders)}] {folder_name}")
        print(f"  기간: {start_date} ~ {end_date}")
        print("-" * 70)
        
        for var_type in variable_types:
            input_dir = os.path.join(folder_path, var_type)
            
            if not os.path.exists(input_dir):
                print(f"  [SKIP] {var_type} 디렉토리 없음")
                continue
            
            tif_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.tif')])
            
            if not tif_files:
                print(f"  [SKIP] {var_type}: TIF 파일 없음")
                continue
            
            print(f"  [{var_type}] {len(tif_files)}개 파일 검사 중...")
            
            var_start = time.time()
            var_processed = 0
            var_skipped = 0
            
            for idx, filename in enumerate(tif_files):
                date_str = filename.split('_')[0]
                
                # 이미 계산된 통계인지 확인
                if (date_str, var_type) in computed_keys:
                    var_skipped += 1
                    continue
                
                input_tif = os.path.join(input_dir, filename)
                
                # 벡터화 통계 계산
                stats_dict = compute_stats_vectorized(
                    input_tif, zone_array, zone_ids, date_str, var_type
                )
                
                if stats_dict is not None:
                    all_new_results.append(stats_dict)
                    computed_keys.add((date_str, var_type))  # 중복 방지
                    var_processed += 1
                    
                    # 진행 상황 출력 (20개마다)
                    if var_processed % 20 == 0:
                        elapsed = time.time() - var_start
                        rate = var_processed / elapsed if elapsed > 0 else 0
                        print(f"    신규 처리: {var_processed}개 ({rate:.1f} files/sec)")
            
            var_elapsed = time.time() - var_start
            
            if var_processed > 0:
                print(f"  [완료] {var_type}: 신규 {var_processed}개, 스킵 {var_skipped}개 "
                      f"({var_elapsed:.1f}초)")
            else:
                print(f"  [완료] {var_type}: 모두 이미 계산됨 (스킵 {var_skipped}개)")
            
            total_processed += var_processed
            total_skipped += var_skipped
    
    # 6. 신규 결과를 DataFrame으로 변환 및 저장
    print(f"\n{'='*70}")
    print("결과 저장")
    print(f"{'='*70}")
    
    if all_new_results:
        print(f"신규 계산된 결과: {len(all_new_results):,}개")
        
        # 딕셔너리 → DataFrame 변환
        print("DataFrame 변환 중...")
        
        rows = []
        for result in all_new_results:
            date_str = result['date']
            var_type = result['variable']
            zone_ids_arr = result['zone_ids']
            
            for i, zid in enumerate(zone_ids_arr):
                rows.append({
                    'cat': int(zid),
                    'date': date_str,
                    'variable': var_type,
                    'mean': float(result['means'][i]),
                    'max': float(result['maxs'][i]),
                    'min': float(result['mins'][i]),
                    'std': float(result['stds'][i]),
                    'count': int(result['counts'][i])
                })
        
        new_results_df = pd.DataFrame(rows)
        print(f"  신규 통계 레코드: {len(new_results_df):,}개")
        
        # 기존 통계와 병합하여 저장 (Long + Wide 형식)
        save_combined_statistics(new_results_df, existing_df, RAW_LONG_CSV, WIDE_PIVOT_CSV)
        
    else:
        print("신규 계산된 결과 없음 (모든 통계가 이미 계산됨)")
        
        if existing_df is not None:
            print(f"기존 통계 유지: {len(existing_df):,}개 레코드")
        else:
            print("[WARNING] 기존 통계도 없고 신규 통계도 없습니다!")
    
    total_elapsed = time.time() - start_time
    
    print(f"\n{'='*70}")
    print("Step 2B 완료! 🎉")
    print(f"{'='*70}")
    print(f"처리 폴더: {len(clipped_folders)}개")
    print(f"신규 처리: {total_processed}개 파일")
    print(f"스킵 (이미 계산됨): {total_skipped}개 파일")
    print(f"총 소요 시간: {total_elapsed:.1f}초 ({total_elapsed/60:.1f}분)")
    
    if total_processed > 0:
        print(f"처리 속도: {total_processed/total_elapsed:.1f} files/sec")
    
    print("\n💡 증분 업데이트 완료:")
    print("  - 여러 기간 폴더 자동 처리")
    print("  - 이미 계산된 통계는 스킵")
    print("  - Long Format CSV: 증분 업데이트용 (각 행 = 측정값)")
    print("  - Wide Format CSV: 모델 학습용 (각 열 = 날짜별 변수)")
    print("="*70)


Step 2B 통계 추출 시작 (증분 업데이트)...


[SCAN] 클립된 LDAPS 폴더 검색 중...
  검색 경로: D:\Landslide\data\processed\gyeongnam\LDAPS
  ✅ 발견: TIF_5179_30m_20190101_20190816_clipped
      → 기간: 20190101 ~ 20190816
  ✅ 발견: TIF_5179_30m_20190817_20200228_clipped
      → 기간: 20190817 ~ 20200228
  ✅ 발견: TIF_5179_30m_20190817_20200228_original
      → 기간: 20190817 ~ 20200228
  ✅ 발견: TIF_5179_30m_20200301_20200930_clipped
      → 기간: 20200301 ~ 20200930

총 4개 폴더 발견

[LOAD] 기존 통계 파일 로드: ldaps_statistics_long_format.csv
  기존 레코드: 56,300,832개
  고유 날짜: 214개
  고유 (날짜, 변수) 조합: 642개

[LOAD] 사면 단위 로드 중...
  사면 개수: 89,529개

[ZONE] Zone Raster 준비 중...
  기존 Zone Raster 사용: zone_raster_gyeongnam_5179_30m.tif
  고유 사면 ID: 87,696개
  격자 크기: (4862, 5040)
  메모리: 93.5 MB

통계 계산 시작

[1/4] TIF_5179_30m_20190101_20190816_clipped
  기간: 20190101 ~ 20190816
----------------------------------------------------------------------
  [acc7d] 228개 파일 검사 중...
    신규 처리: 20개 (0.3 files/sec)
    신규 처리: 40개 (0.3 files/sec)
    신규 처리: 60개 (0.2 fil